# Austrian Emissions Projections

This notebook processes the emissions projections as published by the Umweltbundesamt for use in the energy-scenarios-at Scenario Explorer.

## Latest Projections Update (2025)


### Report

https://www.umweltbundesamt.at/studien-reports/publikationsdetail?pub_id=2615&cHash=cd3136fb362a87beaf854d0dd5bc77bc

> GHG projections and assessment of policies and measures in Austria 2025.
> Reporting under Regulation (EU) 2018/1999. Submission: 15 March 20255
> Wien, 2025
> Reports, Band 0993
> ISBN: 978-3-99004-840-5
> 218 S.

### Data Source

Member States' greenhouse gas (GHG) emission projections 2025
published by the European Environment Agency

DOI: https://doi.org/10.2909/d6938865-05e5-4016-a15c-a09bb898421d

Released under CC-BY according to the European Environment Agency

In [ ]:
file = "GHG_projections2025_final_dataset_2025_with_metadata_sheet"

year = 2025
citation = "Austria's GHG projections and assessment, Report 0993 (UBA, 2025)"
data_source = "https://doi.org/10.2909/d6938865-05e5-4016-a15c-a09bb898421d"

In [ ]:
import pandas as pd
import pyam
import nomenclature

## Projections by gas

In [ ]:
emissions_gas = pd.read_excel(f"source/{file}.xlsx", sheet_name="Database gas")

In [ ]:
emissions_gas = emissions_gas[emissions_gas.CountryCode == "AT"]

In [ ]:
emissions_gas.Category.unique()

In [ ]:
category_mapping = {
    "Total including LULUCF": "",
    "Total excluding LULUCF": " [excl. LULUCF]",
#    "1.A": "|Energy",
    "1.A.1. Energy industries": "|Energy|Supply",
    "1.A.2. Manufacturing industries and construction": "|Energy|Demand|Industry",
    "1.A.3. Transport": "|Energy|Demand|Transportation",
    "1.A.4. Other sectors": "|Energy|Demand|Residential and Commercial",
    "1.A.5. Other": "|Energy|Demand|Other",
    "1.B. Fugitive emissions from fuels": "|Fugitive",
    "2. Industrial processes": "|Industrial Processes",
    "3. Agriculture": "|Agriculture",
    "4. Land Use, Land-Use Change and Forestry (LULUCF, reported emissions and removals) (9)": "|Land Use, Land-Use Change and Forestry",
    "5. Waste": "|Waste",
    "IB.Aviation": "|International Aviation [Memo Item]"
}

In [ ]:
df_list = []

for gas, data in emissions_gas.groupby("Gas"):
    if gas in ["CO2 (kt)", "CH4 (kt)", "N2O (kt)"]:
        gas = gas.split(" ")[0]
        unit = f"kt {gas}"
    else:
        continue

    _df = pyam.IamDataFrame(
        data.drop(columns=["CountryCode", "SubmissionYear", "Gas", "RY calibration", "Reported Value"]),
        model=f"Umweltbundesamt ({year})",
        scenario="Scenario",
        region="Austria",
        variable="Category",
        unit=unit,
        value="Gapfilled",
    )

    variable_mapping = dict(
        [(key, f"Emissions|{gas}{value or ''}") for key, value in category_mapping.items()]    
    )
    
    _df = (
        _df.filter(variable=variable_mapping)
        .rename(variable=variable_mapping)
    )

    if gas != "CO2":
        _df.filter(variable="*International Aviation [Memo Item]", keep=False, inplace=True)
    
    if gas in ["CO2", "CH4"]:
        _df.convert_unit(unit, "Mt" + unit[2:], inplace=True)

    _df.aggregate(f"Emissions|{gas}|Energy|Demand", append=True)
    _df.aggregate(
        f"Emissions|{gas}|Energy",
        append=True,        
    )
    _df.aggregate(
        f"Emissions|{gas}|Energy and Industrial Processes",
        components=[f"Emissions|{gas}|Energy", f"Emissions|{gas}|Industrial Processes"],
        append=True,
    )

    df_list.append(_df)

df_gas = pyam.concat(df_list)

In [ ]:
df_gas

## Kyoto gases

In [ ]:
emissions_ghg = pd.read_excel(f"source/{file}.xlsx", sheet_name="Database")

In [ ]:
emissions_ghg = emissions_ghg[emissions_ghg.CountryCode == "AT"]

In [ ]:
emissions_ghg.Category.unique()

In [ ]:
emissions_ghg.Gas.unique()

In [ ]:
df_list = []

for gas, data in emissions_ghg.groupby("Gas"):

    _df = pyam.IamDataFrame(
        data.drop(columns=["CountryCode", "SubmissionYear", "Gas", "RY calibration", "Reported Value"]),
        model=f"Umweltbundesamt ({year})",
        scenario="Scenario",
        region="Austria",
        variable="Category",
        unit="kt CO2e",
        value="Gapfilled",
    )
    _df.convert_unit("kt CO2e", "Mt CO2e", inplace=True)

    if gas == "Total GHG emissions (ktCO2e)":
        variable_mapping = dict(
            [(key, f"Emissions|Kyoto Gases{value or ''}") for key, value in category_mapping.items()]    
        )
        
        _df = (
            _df.filter(variable=variable_mapping)
            .rename(variable=variable_mapping)
        )

        _df.aggregate("Emissions|Kyoto Gases|Energy|Demand", append=True)
        _df.aggregate(
            "Emissions|Kyoto Gases|Energy",
            append=True,        
        )
        _df.aggregate(
            "Emissions|Kyoto Gases|Energy and Industrial Processes",
            components=["Emissions|Kyoto Gases|Energy", "Emissions|Kyoto Gases|Industrial Processes"],
            append=True,
        )

    else:
        variable_mapping = {
            "Total excluding LULUCF": f"Emissions|Kyoto Gases [{gas[:3]}]",
        }
        
        _df = (
            _df.filter(variable=variable_mapping)
            .rename(variable=variable_mapping)
        )
    df_list.append(_df)

df_ghg = pyam.concat(df_list)

# Finalize and export¶

In [ ]:
df = pyam.concat([df_gas, df_ghg])

In [ ]:
df.rename(
    scenario={
        "WEM": f"With Existing Measures (WEM {year})",
        "WAM": f"With Additional Measures (WAM {year})",
    },
    inplace=True,
)

In [ ]:
definition = nomenclature.DataStructureDefinition("../../definitions/")

In [ ]:
definition.validate(df)

In [ ]:
df.set_meta(name="Citation", meta=citation)
df.set_meta(name="Data Source", meta=data_source)

In [ ]:
df.filter(variable="Emissions|Kyoto Gases").plot()

In [ ]:
df.filter(variable="*Aviation*").timeseries()

In [ ]:
df.to_excel(f"uba_emissions_projections_{year}.xlsx")